# ライブラリ準備

In [54]:
#coding:utf-8
import pylab
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import copy

#画像の読み込み
test = cv2.imread("./Img/input.jpg", cv2.IMREAD_COLOR)#BGRなので気をつける
gray_test = cv2.imread("./Img/input.jpg",cv2.IMREAD_GRAYSCALE)
width = test.shape[1]
height = test.shape[0] 

#画像の書き出し
cv2.imwrite('./Img/test.jpg', test)
cv2.imwrite('./Img/gray_test.jpg',gray_test)

True

# テスト形状取得

In [113]:
#####学習用テスト形状#####
shape = np.zeros((24,2))
bef_shape = np.zeros((24,2))
shape[:,0] = [12,97,24,62,56,42,96,30,125,34,151,48,178,94,174,137,136,158,100,165,62,158,32,136]
shape[:,1] = [49,173,27,143,24,105,46,57,78,31,114,23,150,36,175,73,172,113,150,148,120,172,85,185]

# shape = np.zeros((4,2))
# shape[:,0] = [1,1,1,2]
# shape[:,1] = [3,3,4,4]
# bef_shape = np.zeros((4,2))
# bef_shape[:,0]
gray_shape_arr =list()

black = np.zeros((200,200))
N = int(len(shape[:,0])/2)
for i in range(0,int(N)):
    x = int(shape[2*i][0])
    y = int(shape[2*i+1][0])
    x2 = int(shape[(2*(i+1))%(2*N)][0])
    y2 = int(shape[(2*(i+1)+1)%(2*N)][0])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black_shape0.jpg' ,black)

gray_shape0_img = cv2.imread("./Img/Samples/shape0.jpg",cv2.IMREAD_GRAYSCALE)
gray_shape_arr.append(gray_shape0_img)
cv2.imwrite('./Img/gray_shape0.jpg' ,gray_shape_arr[0])

black = np.zeros((200,200))
N = int(len(shape[:,1])/2)
for i in range(0,int(N)):
    x = int(shape[2*i][1])
    y = int(shape[2*i+1][1])
    x2 = int(shape[(2*i+2)%(2*N)][1])
    y2 = int(shape[(2*i+3)%(2*N)][1])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black_shape1.jpg' ,black)

gray_shape1_img = cv2.imread("./Img/Samples/shape1.jpg",cv2.IMREAD_GRAYSCALE)
gray_shape_arr.append(gray_shape1_img)
cv2.imwrite('./Img/gray_shape1.jpg' ,gray_shape_arr[1])

#####平均形状取得######
black = np.zeros((200,200))
shape_ave = (shape[:,0]+shape[:,1])/2 #平均形状
for i in range(0,int(N)):
    x = int(shape_ave[2*i])
    y = int(shape_ave[2*i+1])
    x2 = int(shape_ave[(2*(i+1))%(2*N)])
    y2 = int(shape_ave[(2*(i+1)+1)%(2*N)])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black_shape_ave.jpg' ,black)




True

# 外観の正規化

In [75]:
gray_rot_arr = list()#正規化された外観

for j in range(0,len(gray_shape_arr)):
    N = int(len(shape[:,j])/2)
    #平均形状の重心
    g_ave = np.array([0,0])
    g_ave = np.sum(shape_ave[::2])/N,np.sum(g_ave[1::2])/N
    #形状の重心
    g_shape = np.array([0,0])
    g_shape = np.sum(shape[::2,j])/N,np.sum(shape[1::2,j])/N
    shape[::2,j] -= g_shape[0]
    shape[1::2,j] -= g_shape[1]
    #パラメータ計算
    param_a = np.dot(shape[:,j],shape_ave[:])/np.dot(shape[:,j],shape[:,j])
    param_b = 0
    loop = int(shape[:,j].shape[0]/2)
    for i in range(0,loop):
        x1_ = shape[2*i,j]
        y1_ = shape[2*i+1,j]
        x2_ = shape_ave[2*i]
        y2_ = shape_ave[2*i+1]
        param_b += (x1_*y2_-x2_*y1_ )
    param_b /= np.dot(shape[:,j],shape[:,j])
    param_s = (param_a**2+param_b**2)**(1/2)
    param_theta = np.arctan2(param_b,param_a)
    print("拡大率s:",param_s,",回転角θ:",param_theta)
    # 回転変換行列の算出
    rotation_matrix = cv2.getRotationMatrix2D((100,100), param_theta*180/math.pi ,param_s)#現在(200,200)のみにつくってる
    size = tuple((200,200))
    img_rot = cv2.warpAffine(gray_shape_arr[j], rotation_matrix, size, flags=cv2.INTER_CUBIC)
    gray_rot_arr.append(img_rot)
    cv2.imwrite("./Img/gray_shape"+str(j)+"_rot.jpg" ,img_rot)

拡大率s: 0.92588550143 ,回転角θ: -0.432142521814
拡大率s: 0.892325607835 ,回転角θ: 0.415388246076


# 平均外観の取得

In [115]:
appearance_ave = np.zeros((200,200))
for val in gray_rot_arr:
    appearance_ave += val
appearance_ave /=  len(gray_rot_arr)
cv2.imwrite("./Img/appearance_ave.jpg" ,appearance_ave)


True

# 主成分分析
http://www.cvlab.cs.tsukuba.ac.jp/index.php?plugin=attach&pcmd=open&file=imgrec2016-handout-5.pdf&refer=2016%C7%AF%C5%D9%20%B2%E8%C1%FC%C7%A7%BC%B1%B9%A9%B3%D8

In [140]:
shape_sample_num  = shape.shape[1]

S = np.zeros((2*N,2*N))
for i in range(0,2*N):
    for j in range(0,2*N):
        for n in range(0,shape_sample_num):
            S[i,j] += (shape[i][n]-shape_ave[i])*(shape[j][n]-shape_ave[j])
S /= shape_sample_num-1

#輝度値共分散行列
A_mat = np.zeros((200,200))
for i in range(0,200):
    for j in range(0,200):
        for n in range(0,shape_sample_num):
            A_mat[i,j] += np.dot(gray_rot_arr[n][:][j]-appearance_ave[:][j],gray_rot_arr[n][i][:]-appearance_ave[i][:])
A_mat /= shape_sample_num-1


# ma = np.max(S)
# S /= ma
# S *= 255
# cv2.imwrite('./Img/s.jpg' ,S)
# ma = np.max(A_mat)
# A_mat /= ma
# A_mat *= 255
# cv2.imwrite('./Img/a_mat.jpg' ,A_mat)

True

In [379]:
la, v = np.linalg.eigh(S)
print(u"固有値："+str(la))
print(u"固有ベクトル："+str(v))
# la[la<10**(-20)] = 0 #小さすぎる値は0 として近似
print(la) #最終的な固有値

固有値：[ -4.50619187e-16  -6.26676482e-17   2.06759907e-15   1.05000000e+01]
固有ベクトル：[[-0.76499252 -0.30646414  0.36109554  0.43643578]
 [-0.17812879  0.02753671 -0.88149627  0.43643578]
 [ 0.61873334 -0.39202786  0.18684697  0.65465367]
 [ 0.01502131  0.86696922  0.24013027  0.43643578]]
[ -4.50619187e-16  -6.26676482e-17   2.06759907e-15   1.05000000e+01]


In [380]:
eigen_id = np.argsort(la)[::-1]
la = la[eigen_id]
v = v[:,eigen_id]
print(la)
print(v)

[  1.05000000e+01   2.06759907e-15  -6.26676482e-17  -4.50619187e-16]
[[ 0.43643578  0.36109554 -0.30646414 -0.76499252]
 [ 0.43643578 -0.88149627  0.02753671 -0.17812879]
 [ 0.65465367  0.18684697 -0.39202786  0.61873334]
 [ 0.43643578  0.24013027  0.86696922  0.01502131]]


In [381]:
la_sum = np.sum(la)
need_la = 0 #必要固有値の和
need_la_num = 1 #必要固有値数-1
for val in la:
    need_la += val
    if(la_sum*0.98 < need_la):
        break
    else:
        need_la_num +=1
print(need_la_num)

1


In [382]:
P = np.empty((N*2,need_la_num))
for j in range(0,need_la_num):
    P[:,j] = v[:,j]
b = np.empty((need_la_num))
print(P)
print(b)

[[ 0.43643578]
 [ 0.43643578]
 [ 0.65465367]
 [ 0.43643578]]
[ 10.5]


In [383]:
param=2#(±3までらしいが、、、)
for i in range(0,len(b)):
    b[i] = la[i]**(1/2)*param
# print(P.shape,b.shape,result.shape)
# print(P)
# print(b)
result = shape_ave+np.dot(P,b)

# print(result)
# 輪郭線のテスト
black_result = np.zeros((200,200))
N = int(len(shape[:,0])/2)
for i in range(0,int(N)):
    x = int(result[2*i])
    y = int(result[2*i+1])
    x2 = int(result[(2*(i+1))%(2*N)])
    y2 = int(result[(2*(i+1)+1)%(2*N)])
    print(x,y,x2,y2)
    black_result = cv2.line(black_result, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black_result.jpg' ,black_result)

4 4 6 5
6 5 4 4


True

In [26]:
#test
hoge= np.zeros((4,2))
fuga_  = np.zeros((4,2))
fuga_[:,0] =  [1,1,1,2]
fuga_[:,1] = [3,3,4,4]
hoge[:,0] = copy.deepcopy(fuga_[:,0])
hoge[:,1] = copy.deepcopy(fuga_[:,1])
print("移動前:\n",hoge)
#重心とる
N = int(len(hoge[:,0])/2)
g1_ = np.array([0,0])
g1_ = np.sum(hoge[::2,0])/N,np.sum(hoge[1::2,0])/N
g2_ = np.array([0,0])
g2_ = np.sum(hoge[::2,1])/N,np.sum(hoge[1::2,1])/N
hoge[::2,0] -= g1_[0]
hoge[1::2,0] -= g1_[1]
hoge[::2,1] -= g2_[0]
hoge[1::2,1] -= g2_[1]
print(g2_)
print("移動後:\n",hoge)
#パラメータ計算
a_ = np.dot(hoge[:,0],hoge[:,1])/np.dot(hoge[:,0],hoge[:,0])
b_ =0
loop = int(hoge[:,0].shape[0]/2)
for i in range(0,loop):
    x1_ = hoge[2*i,0]
    y1_ = hoge[2*i+1,0]
    x2_ = hoge[2*i,1]
    y2_ = hoge[2*i+1,1]
    b_ += (x1_*y2_-x2_*y1_ )
b_ /= np.dot(hoge[:,0],hoge[:,0])
s_ = (a_**2+b_**2)**(1/2)
theta_ = np.arctan2(b_,a_)
print("s:",s_,"θ:",theta_)


移動前:
 [[ 1.  3.]
 [ 1.  3.]
 [ 1.  4.]
 [ 2.  4.]]
(3.5, 3.5)
移動後:
 [[ 0.  -0.5]
 [-0.5 -0.5]
 [ 0.   0.5]
 [ 0.5  0.5]]
s: 1.41421356237 θ: -0.785398163397


NameError: name 'center' is not defined

In [385]:
sA_ =  np.zeros((2,2))
sA_[:,0] = [s_*np.cos(theta_),-s_*np.sin(theta_)]
sA_[:,1] = [s_*np.sin(theta_), s_*np.cos(theta_)]

ans_ = [0,0,0,0]
for i in range(0,loop):
    aho = [hoge[2*i,0],hoge[2*i+1,0]]
    ans_[2*i],ans_[2*i+1] = np.dot(aho,sA_) +g2_




In [386]:
ans_,fuga_[:,1]

([3.0, 3.0, 4.0, 4.0], array([ 3.,  3.,  4.,  4.]))